# API Demo: Search, Order, and Download Capella's Rotterdam Data Set

Capella has collected two AOIs - 10 x 5 km each. Each AOI will be covered from 8am to 8pm over two days, 
6 times each day to simulate roughly two hour revisit at very high resolution. 
The main features in the AOIs are: the port, ships (tankers and commercials), shipping containers, cars,
parking lot, stadium, train rails, floating oil tanks, downtown, vegetated areas. 
Imagery is be generated using backprojection in map coordinates.

In [ ]:
# Required libraries:
# requests
# json
# urllib

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# JSON utility function
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.data.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/search'
orders = '/orders/'

# Load my username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

### Get and Print Access Token

In [ ]:
# Get the token 
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# Print the token
print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}

### Print Available Collections

In [ ]:
# See what collections are available
r = requests.get(URL + collections, headers=headers)

# Print results
p(r.json())


### Post Search Filters for Rotterdam data set, Print the Results

In [ ]:
# Filters 
filters = {
  "time": "2019-08-01T00:00:00Z/2019-08-31T12:31:12Z",
  "limit": 50, # overwrite the default pagination limit of 10, adjust as necessary
  "collections": ["rotterdam-aerial"], # specify the desired collection
  "query": {"sar:product_type": {"eq":"GEO"},
            "sar:polarizations": {"eq":"HH"}},  
  "sort": [{"field": "dtr:start_datetime"}],
  
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Print the results
p(r.json())

### Make and Post an Order

In [ ]:
# Build the Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)

myorder = {"Items": granulelist}

# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

### Get the STAC records with the signed URLs using the /download endpoint, Print the Result

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
p(r.json())


### Download the Results

In [ ]:
features = r.json()

# Local directory to save data; Update to preference
basefp = 'C:/data/rotterdam/' 

for feature in features:

    filepath = feature["assets"]["HH"]["href"]

    filename = filepath[filepath.rfind("/")+1:]

    sep = "?"
    truncname = filename.split(sep, 1)[0]
    
    outfp = basefp + truncname
    
    import urllib
    f = urllib.request.urlretrieve(filepath, outfp)
